In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
import cv2

from tensorflow.keras.applications.resnet_v2 import ResNet50V2, decode_predictions, preprocess_input

In [16]:
import pandas as pd 

batch_size = 32
img_height = 48
img_width = 48

data_dir = pd.read_csv("train_set.csv", sep=";") 

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

TypeError: listdir: path should be string, bytes, os.PathLike or None, not DataFrame

In [7]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (img_height, img_width, 3),
                  include_top=False,    # false porque ahora la vamos a entrenar nosotros. No nos cogemos todos los pesos
                  weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False # el modelo no se entrena, pero ya tiene sus pesos

    
##### FULLY CONNECTED LAYER #####
# creamos una pequeña red al final del modelo para entrenarlo para nuestros datos

# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x) # esta x se refiere a la de antes

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

# Aquí creamos un modelo concatenado, el ya existente y nuestras nuevas capas
model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['acc'])

58892288/58889256 [==============================] - 5s 0us/step


In [9]:
vgghist = model.fit(train_ds,
                    validation_data = val_ds,
                    epochs = 10,
                    verbose=1)

Epoch 1/10
155/155 [==============================] - 22s 142ms/step - loss: 0.3807 - acc: 0.8290 - val_loss: 0.7065 - val_acc: 0.7028
Epoch 2/10
155/155 [==============================] - 23s 145ms/step - loss: 0.3539 - acc: 0.8419 - val_loss: 0.7162 - val_acc: 0.7263
Epoch 3/10
155/155 [==============================] - 23s 148ms/step - loss: 0.3345 - acc: 0.8488 - val_loss: 0.8212 - val_acc: 0.7142
Epoch 4/10
155/155 [==============================] - 21s 138ms/step - loss: 0.3040 - acc: 0.8666 - val_loss: 0.7632 - val_acc: 0.7077
Epoch 5/10
155/155 [==============================] - 22s 144ms/step - loss: 0.2948 - acc: 0.8780 - val_loss: 0.7859 - val_acc: 0.7263
Epoch 6/10
155/155 [==============================] - 23s 147ms/step - loss: 0.2749 - acc: 0.8865 - val_loss: 0.8267 - val_acc: 0.7190
Epoch 7/10
155/155 [==============================] - 23s 149ms/step - loss: 0.2632 - acc: 0.8887 - val_loss: 0.8895 - val_acc: 0.7158
Epoch 8/10
155/155 [==============================] - 2

In [11]:
# Predecimos primero los datos de entrenamiento
trainPredict = model.predict(train_ds)

# y luego el de test
#testPredict = model.predict(test_ds)

# concatenamos antes
#predicted = np.concatenate((trainPredict,testPredict),axis=0)